# 🌵 AI Comic — "The Misguided Enthusiasm of the Little Cactus Boy"
### Llama 3.1 8B via Groq · FLUX.1-dev · LongCLIP · 41 Scenes · H100

**Story by the user.**
- **Llama 3.1 8B Instruct** (Groq API + LangChain) — dialogue, 0 VRAM
- **FLUX.1-dev** — images, 2D cartoon style ONLY
- **LongCLIP zer0int/LongCLIP-GmP-ViT-L-14** — evaluation, 248-token limit

```
Groq API       :  0 GB  → dialogue generation (API call)
FLUX.1-dev     : ~33 GB → image generation
LongCLIP       :  ~1 GB → evaluation


# 📦 STEP 0 — INSTALL

In [ ]:
%%capture
!pip install -q "diffusers>=0.32.0" "transformers>=4.44.0" accelerate
!pip install -q Pillow reportlab sentencepiece protobuf
!pip install -q langchain-groq langchain-core          # Groq + LangChain
!pip install -q open-clip-torch                        # LongCLIP
print('✅ Installation complete')


# 🔑 STEP 0B — API KEYS & HUGGING FACE LOGIN

### 1. Groq API key (for Llama 3.1 8B dialogue)
Get a free key at https://console.groq.com → API Keys → Create

Add to **Colab Secrets** (🔑 left sidebar):
- Name: `GROQ_API_KEY`  |  Value: your key (`gsk_...`)

> ⚠️ Never paste API keys directly in code — always use Colab Secrets.

---

### 2. HuggingFace login (for FLUX.1-dev)
→ https://huggingface.co/black-forest-labs/FLUX.1-dev
Click **"Agree and access repository"** (instant approval, free gate)

→ https://huggingface.co/settings/tokens — create **Read** token

Add to **Colab Secrets**:
- Name: `HF_TOKEN`  |  Value: your token (`hf_...`)

> The 4-bit quantized version at `diffusers/FLUX.2-dev-bnb-4bit` uses the same gate.
> LongCLIP is fully open — no login needed.


In [ ]:
from huggingface_hub import login
from google.colab import userdata

# ── 1. Groq API key ───────────────────────────────────────────────
try:
    GROQ_API_KEY = userdata.get('put the name of your Groq API key here')
    print(f'✅ Groq API key loaded from Colab Secrets')
    print(f'   Key preview: {GROQ_API_KEY[:8]}...{GROQ_API_KEY[-4:]}')
except Exception:
    # Fallback: paste directly (delete after confirming it works)
    GROQ_API_KEY = 'gsk_Enter_Yours_Groq_API_Value_Here'
    if GROQ_API_KEY == 'gsk_Enter_Yours_Groq_API_Value_Here':
        print('⚠️  No Groq key found.')
        print('   Add GROQ_API_KEY to Colab Secrets (🔑 left sidebar)')
        print('   or replace the placeholder above.')
        print('   Get a free key at: https://console.groq.com')
    else:
        print('✅ Groq API key loaded from cell (remember to delete it)')

# ── 2. HuggingFace login (FLUX.1-dev only) ────────────────────────
try:
    hf_token = userdata.get('put the name of your Hugging Face API key here')
    login(token=hf_token, add_to_git_credential=False)
    print('✅ HuggingFace logged in via Colab Secrets')
except Exception:
    HF_TOKEN = 'hf_Enter_Yours_Hugging_Face_API_Value_Here'
    if HF_TOKEN == 'hf_Enter_Yours_Hugging_Face_API_Value_Here':
        print('⚠️  No HF token found.')
        print('   Add HF_TOKEN to Colab Secrets or replace placeholder above.')
        print('   Accept FLUX gate: https://huggingface.co/black-forest-labs/FLUX.2-dev')
        print('   Token page      : https://huggingface.co/settings/tokens')
    else:
        login(token=HF_TOKEN, add_to_git_credential=False)
        print('✅ HuggingFace logged in (remember to delete token from cell)')


# ⚙️ STEP 1 — IMPORTS & SETUP

In [ ]:
import torch, os, json, gc, re, numpy as np
from PIL import Image, ImageDraw, ImageFont
from datetime import datetime
from IPython.display import display

import open_clip                                               # LongCLIP
from diffusers import FluxPipeline                            # FLUX.1-dev
from langchain_groq import ChatGroq                           # Groq + LangChain
from langchain_core.messages import SystemMessage, HumanMessage
from transformers import (
    CLIPTokenizer,
    GPT2LMHeadModel, GPT2TokenizerFast                        # perplexity
)
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas as pdf_canvas
from reportlab.lib import colors
from reportlab.platypus import (SimpleDocTemplate, Paragraph, Spacer,
                                 Table, TableStyle, PageBreak)
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.units import inch

os.makedirs('comic/images', exist_ok=True)
os.makedirs('comic/evaluation', exist_ok=True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'✅ Device: {device}')
if device == 'cuda':
    vram = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f'   GPU : {torch.cuda.get_device_name(0)}')
    print(f'   VRAM: {vram:.1f} GB')
    print()
    print('  Groq API (Llama 3.1 8B) :  0 GB  — runs remotely')
    print('  FLUX.1-dev              : ~33 GB')
    print('  LongCLIP                :  ~1 GB')
    print('  H100 (80 GB)            : ✅')


# 🔑 STEP 2 — TOKEN COUNT HELPER

FLUX.1-dev uses a **T5-XXL text encoder** (up to 512 tokens).
The full rich scene prompt is processed as one unified description.

**CRITICAL:** FLUX does **not** use negative prompts — the architecture ignores them.
To enforce flat 2D cartoon style and prevent realistic/3D rendering,
we must repeat "hand-drawn, flat, 2D cartoon" language **inside the positive STYLE**.


In [ ]:
clip_tok_check = CLIPTokenizer.from_pretrained('openai/clip-vit-large-patch14')

def token_count(text):
    return len(clip_tok_check.encode(text, add_special_tokens=False))

print('Token counter ready.')
print()
print('  Text encoder limits in this pipeline:')
print('  ✅ image_prompts  → FLUX T5-XXL   (512 tok)')
print('  ✅ scene_context  → LongCLIP      (248 tok)')


# 🖼️ STEP 3 — UPLOAD STYLE REFERENCE (optional)

In [ ]:
from google.colab import files
import io
print('📤 Upload 1-3 style reference images (optional)')
uploaded = files.upload()
if uploaded:
    for idx, (name, data) in enumerate(uploaded.items()):
        img = Image.open(io.BytesIO(data)).convert('RGB').resize((512,512))
        img.save(f'comic/style_ref_{idx:02d}.png')
        print(f'  ✅ {name}')
        display(img.resize((180,180)))
else:
    print('ℹ️  No upload — default style used')


# 🌵 STEP 4 — CHARACTERS

**CACTUS_BOY**: green cactus child, cartoon eyes, spines, red backpack

**FATHER_CAC**: tall green cactus adult, cartoon eyes, thick spines, glasses

**MONKEY_ORCHID**: orchid flower, monkey-face petal pattern, purple/white

**RAIN_PLANT**: tropical plant, large green leaves, water droplets


Style: **Pure flat 2D vintage disney cartoon** — hand-drawn vintage comic look....etc.



In [ ]:
# ── Visual anchors — repeated exactly in every prompt ──────────────

CACTUS_BOY = (
    'small green cactus child character with round body, big round expressive cartoon eyes with white highlights, '
    'tiny stubby cartoon arms with simple rounded hands, body completely covered in short green spines, '
    'wearing bright cherry-red school backpack with visible black straps and silver buckles, '
    'simple clean 2D flat cartoon design with bold black outlines, completely flat coloring with no shading or depth'
)
FATHER_CAC = (
    'tall mature green cactus adult character with elongated cylindrical body, kind warm cartoon eyes behind round black-framed glasses, '
    'thick prominent spines covering entire body surface uniformly, gentle reassuring fatherly cartoon expression with slight smile, '
    'stands approximately twice the height of child character, '
    'simple clean 2D flat cartoon design with bold black outlines, completely flat coloring with no shading or depth'
)
MONKEY_ORC = (
    'orchid flower character with distinct monkey face pattern clearly visible on purple and white petals, '
    'friendly warm cartoon expression showing through flower face design, small rounded green leaf arms extending from stem base, '
    'simple stem structure, '
    'simple clean 2D flat cartoon design with bold black outlines, completely flat coloring with no shading or depth'
)
RAIN_PLANT = (
    'tall lush tropical plant character with multiple large broad shiny green leaves spreading outward from central stem, '
    'small crystal-clear water droplets visibly sitting on leaf surfaces, cheerful smiling cartoon face in center, '
    'vine-like green arms with rounded bulbous ends extending from sides, '
    'simple clean 2D flat cartoon design with bold black outlines, completely flat coloring with no shading or depth'
)

# Compact versions for multi-character scenes
BOY_S  = 'cartoon cactus boy green spines red backpack 2D flat simple black outlines'
DAD_S  = 'cartoon cactus father tall glasses thick spines 2D flat simple black outlines'
MONK_S = 'cartoon orchid flower monkey face purple white 2D simple black outlines'
RAIN_S = 'cartoon tropical plant big green leaves water droplets 2D flat simple black outlines'

# ── STYLE:comic specification ──

STYLE = (
    # === CORE STYLE FOUNDATION ===
    'hand-drawn flat 2D cartoon comic panel illustration in pure classic vintage Disney comic book style from 1950s-1960s, '

    # === COLOR RENDERING SPECIFICATION ===
    'completely flat bright vivid solid colors filled inside black outlines like traditional animation cels, '
    'absolutely zero gradients anywhere in the image, no color transitions, no blending, no soft edges, '
    'zero shading of any kind, no shadows cast by characters or objects, no lighting effects, no highlights, no reflections, '

    # === CHARACTER RENDERING SPECIFICATION ===
    'expressive cartoon eyes clearly showing emotions through shape and pupil placement, '
    'simplified cartoon anatomy using basic rounded geometric shapes,'
    'characters drawn with perfectly consistent proportions and design across all panels, same height ratios, same body shapes, '
    'every character drawn full body visible from head to toe in complete view,'
    'characters standing on clearly defined ground line or floor surface, proper weight and grounding, '

    # === BACKGROUND AND ENVIRONMENT SPECIFICATION ===
    'richly detailed hand-drawn background with all environment elements, props, and scenery fully illustrated and complete, '
    'backgrounds drawn in same flat 2D style as characters with bold black outlines and flat colors, '
    'clear spatial separation between foreground characters and background environment using overlapping outlines only, '
    'all furniture, objects, plants, rocks, buildings drawn with same level of detail and bold outline treatment, '
    'complete scene context showing walls, floors, skies, landscapes fully illustrated edge to edge, '

    # === LIGHTING AND ATMOSPHERE SPECIFICATION ===
    'even consistent lighting across entire scene with no directional light source visible, '
    'no cast shadows on ground or walls, no character self-shadowing, '
    'flat uniform brightness throughout image, no dark areas or light spots, '
    'clear readable visibility of all scene elements equally, '

)

CHARS = {
    'boy':    {'name': 'Cactus Boy',    'desc': CACTUS_BOY, 'seed': 42},
    'dad':    {'name': 'Father Cactus', 'desc': FATHER_CAC, 'seed': 42},
    'orchid': {'name': 'Monkey Orchid', 'desc': MONKEY_ORC, 'seed': 42},
    'rain':   {'name': 'Rain Plant',    'desc': RAIN_PLANT, 'seed': 42},
}

print('✅ Characters with disney vintage comic style detailed:')
for k,c in CHARS.items():
    tc = token_count(c['desc'])
    print(f'  {c["name"]:15s}: {tc:>2} tokens | {c["desc"][:80]}...')
print(f'\n  STYLE    : {token_count(STYLE)} tokens (ultra-detailed specification)')


# 📖 STEP 5 — STORY OUTLINE FOR LLAMA 3.1 8B (via Groq)

Structured story outline that Llama 3.1 8B (Groq API) will compose into dialogues.
Each entry: `act`, `scene_context` (sent to Llama), `speaker`, `img` (for FLUX),
and `fallback`. Image prompts are richly detailed for FLUX T5-XXL (512 tokens).


In [ ]:
# Pattern: CHARACTER, PROP visible, ACTION + emotion, FULL BACKGROUND, lighting, STYLE

STORY_OUTLINE = [

  # ══════ ACT 1: MORNING AND CAR RIDE ═══════════════════════════════
  {
    'chars': ['boy'],
    'act':   'Morning',
    'scene_context':
      'A small cactus boy wakes up excited in his desert bedroom, '
      'jumps out of bed and picks up his school backpack for the first day of school.',
    'speaker': 'Cactus Boy',
    'img':  CACTUS_BOY + ', '
            'cozy desert bedroom with sandy orange walls and small round window, '
            'golden early morning sunlight streaming through window casting warm glow, '
            'cactus-shaped bed with rumpled yellow blanket, '
            'bright red school backpack sitting on the floor beside the bed, '
            'leaps out of bed with arms raised high in excitement, '
            'small cactus potted plant on windowsill, desert landscape outside window, '
            + STYLE,
    'fallback': 'Today is my first school day!'
  },
  {
    'chars': ['boy', 'dad'],
    'act':   'Morning',
    'scene_context':
      'Father cactus helps his son put on the school backpack in the desert home, '
      'smiling warmly, the son looks happy and ready.',
    'speaker': 'Father Cactus',
    'img':  DAD_S + ' kneels down gently to help '
            + BOY_S + ' put on bright red school backpack, '
            'warm cozy desert home interior with sandy terracotta walls and cactus artwork, '
            'golden morning light from arched window fills the room, '
            'both smiling warmly with love and excitement, '
            'wooden shelf with cactus figurines in background, '
            'cheerful domestic scene, loving father-son moment, '
            + STYLE,
    'fallback': 'Ready? This is a big day!'
  },
  {
    'chars': ['boy', 'dad'],
    'act':   'Car Ride',
    'scene_context':
      'Father cactus drives his son through the desert in a car, '
      'sandy landscape with cacti and rocks outside the windows.',
    'speaker': 'Cactus Boy',
    'img':  DAD_S + ' drives round green car with cactus design, '
            + BOY_S + ' sits in passenger seat looking out the window with wide eyes, '
            'viewed from outside the car window, '
            'wide sandy desert road stretching into the horizon, '
            'tall saguaro cacti and orange sandstone rocks line both sides of the road, '
            'bright blue cloudless sky above, '
            'warm sunny morning atmosphere, long road ahead to school, '
            + STYLE,
    'fallback': 'School is far away, Father!'
  },
  {
    'chars': ['boy', 'dad'],
    'act':   'Car Ride',
    'scene_context':
      'Father cactus gives advice to his son while driving: there are many kinds of plants '
      'you have never seen before, and not all environments suit every plant.',
    'speaker': 'Father Cactus',
    'img':  DAD_S + ' turns head slightly to speak wisely while gripping steering wheel, '
            + BOY_S + ' sits in passenger seat listening with a thoughtful expression, '
            'car interior view with dashboard and front windshield, '
            'desert road ahead visible through windshield, '
            'father gestures with one hand to emphasize his point, '
            'warm afternoon light fills the car interior, '
            + STYLE,
    'fallback': 'Not all plants suit every place.'
  },
  {
    'chars': ['boy', 'dad'],
    'act':   'Car Ride',
    'scene_context':
      'Father cactus continues his advice as they near the school: every plant has its own '
      'advantages, everything is created with precise measure, be content with who you are.',
    'speaker': 'Father Cactus',
    'img':  DAD_S + ' places a gentle hand on ' + BOY_S + ' shoulder while driving, '
            'school building with colorful sign visible through car windshield ahead, '
            'father has a warm sincere smile, cactus boy looks up at his father with trust, '
            'morning sunlight from windshield lights their faces warmly, '
            'touching moment of parental wisdom near the school, '
            + STYLE,
    'fallback': 'Be proud of who you are, son.'
  },

  # ══════ ACT 2: SCHOOL ARRIVAL ══════════════════════════════════════
  {
    'chars': ['boy'],
    'act':   'School Arrival',
    'scene_context':
      'The cactus boy arrives at school and sees the entrance full of plants of all kinds '
      'he has never seen before, from many different environments.',
    'speaker': 'Cactus Boy',
    'img':  CACTUS_BOY + ', '
            'stands at colorful school entrance gate with open mouth in amazement, '
            'crowd of diverse plant students: tall tropical rain plants, '
            'fluffy snow-covered plants, rocky mountain plants, leafy coastal plants, '
            'muddy swamp plants, all walking through the gate together, '
            'vibrant school facade with welcome sign in background, '
            'cactus boy looks tiny among the diverse crowd, eyes wide with wonder, '
            + STYLE,
    'fallback': 'So many plants I never saw!'
  },
  {
    'chars': ['boy'],
    'act':   'School Arrival',
    'scene_context':
      'The cactus boy walks through the school hallway amazed, seeing plants from every '
      'environment: desert, mountain, rain, snow, coast, mud.',
    'speaker': 'Cactus Boy',
    'img':  CACTUS_BOY + ', '
            'walks slowly through wide colorful school hallway with mouth open in awe, '
            'classroom doors on both sides with signs reading DESERT, MOUNTAIN, RAIN, SNOW, COAST, MUD, '
            'different plant students visible through open doorways, '
            'bright cheerful hallway with colorful lockers, banners and potted plants, '
            'cactus boy pauses in the middle turning to look at everything, '
            + STYLE,
    'fallback': 'There are so many classrooms!'
  },
  {
    'chars': ['boy'],
    'act':   'School Arrival',
    'scene_context':
      'The cactus boy peeks through the glass window of the rainy-environment classroom, '
      'seeing lush tropical plants inside a wet misty room, fascinated.',
    'speaker': 'Cactus Boy',
    'img':  CACTUS_BOY + ', '
            'stands on tiptoes pressing his small face flat against glass classroom window, '
            'through the window: lush green tropical rain classroom visible, '
            'misty humid air inside with water dripping from ceiling, '
            'large-leafed tropical plants sitting at desks inside, '
            'cactus boy has wide longing eyes and small hands pressed on glass, '
            'corridor hallway background behind him, warm vs cool lighting contrast, '
            + STYLE,
    'fallback': 'I wish I could go in there!'
  },
  {
    'chars': ['boy'],
    'act':   'Desert Classroom',
    'scene_context':
      'The cactus boy goes to his own desert classroom and sits down, '
      'looking bored because all the plants look just like him.',
    'speaker': 'Cactus Boy',
    'img':  CACTUS_BOY + ', '
            'slumps at a sandy-colored wooden desk in desert classroom, '
            'classroom has sandy floor, cactus-patterned wallpaper, warm orange lighting, '
            'rows of other green cactus students at desks all around, '
            'cactus boy has chin resting on tiny arm looking bored and deflated, '
            'teacher cactus at chalkboard in background with desert diagram, '
            'all students look nearly identical, emphasizing the monotony, '
            + STYLE,
    'fallback': 'This class is so boring here.'
  },
  {
    'chars': ['boy'],
    'act':   'Desert Classroom',
    'scene_context':
      'In the desert classroom, the cactus boy keeps looking out at the hallway toward '
      'the rain classroom, distracted and wishing he could go there.',
    'speaker': 'Cactus Boy',
    'img':  CACTUS_BOY + ', '
            'sits at desk in sandy desert classroom completely distracted, '
            'stares through open classroom door into hallway, '
            'far down the hallway the rain classroom door glows with green misty light, '
            'other cactus students study around him unaware, '
            'teacher writes on chalkboard but cactus boy ignores it, '
            'dreamy distracted expression, thought bubble shows tropical green plants, '
            + STYLE,
    'fallback': 'I want to be with them...'
  },

  # ══════ ACT 3: RECESS DRAMA ════════════════════════════════════════
  {
    'chars': ['boy'],
    'act':   'Recess',
    'scene_context':
      'The school recess bell rings loudly, all the plant students rush out of classrooms '
      'into the school yard happily.',
    'speaker': 'Cactus Boy',
    'img':  CACTUS_BOY + ', '
            'looks up suddenly alert and excited as loud school bell rings on wall, '
            'classroom doors burst open all at once down the hallway, '
            'flood of diverse plant students of all shapes rush out happily cheering, '
            'cactus boy stands up from desk with determined expression and plan on his face, '
            'bright school hallway with sunlight streaming in from windows, '
            + STYLE,
    'fallback': 'Recess! Now is my chance!'
  },
  {
    'chars': ['boy'],
    'act':   'Recess',
    'scene_context':
      'During recess, the cactus boy walks toward the rain classroom door confidently, '
      'determined to talk to the tropical rain plants inside.',
    'speaker': 'Cactus Boy',
    'img':  CACTUS_BOY + ', '
            'walks with confident determined stride down empty school hallway, '
            'heading directly toward the rain classroom door ahead, '
            'rain classroom door has misty green glow and water trickling underneath it, '
            'hallway is quiet and empty during recess, '
            'cactus boy has a bold expression with fists clenched by sides, '
            'dramatic perspective with door getting closer ahead, '
            + STYLE,
    'fallback': 'I will talk to them right now!'
  },
  {
    'chars': ['boy', 'rain'],
    'act':   'Rain Classroom',
    'scene_context':
      'The cactus boy enters the rain classroom. It is full of water and mist. '
      'The tropical plants all stare at him, shocked by his appearance.',
    'speaker': 'Rain Plant',
    'img':  BOY_S + ' stands at open classroom door dripping with immediate humidity, '
            + RAIN_S + ' and all other tropical plants turn to stare in shock why he is here, '
            'rain classroom interior: wet floor with puddles, mist in the air, '
            'water droplets falling from lush plant ceiling decorations, '
            'all rain plants at their desks turn heads toward the door with surprised faces, '
            'cactus boy looks small and out-of-place among the lush green tropical plants, '
            'stark contrast between dry cactus and wet tropical environment, '
            + STYLE,
    'fallback': 'Why is a cactus in our room?!'
  },
  {
    'chars': ['boy', 'rain'],
    'act':   'Rain Classroom',
    'scene_context':
      "The tropical rain plants laugh and point at the cactus boy's thorns, "
      'saying he cannot get close to them because of his spines.',
    'speaker': 'Rain Plant',
    'img':  RAIN_S + ' and several other tropical plants laugh loudly pointing at '
            + BOY_S + ' standing alone, '
            'cactus boy looks down at his own spines with a hurt sad expression, '
            'tropical plants lean away dramatically from the spines as if scared, '
            'some point and cover their mouths laughing, '
            'wet rain classroom background with puddles on the floor, '
            'cactus boy shrinks smaller with sadness surrounded by laughing plants, '
            + STYLE,
    'fallback': 'Your thorns hurt us! Go away!'
  },
  {
    'chars': ['boy'],
    'act':   'Rain Classroom',
    'scene_context':
      'The cactus boy walks out of the rain classroom feeling very sad, '
      'shoulders down, hurt by the laughter of the rain plants.',
    'speaker': 'Cactus Boy',
    'img':  CACTUS_BOY + ', '
            'shuffles out of rain classroom door into hallway, shoulders slumped low, '
            'head bowed down with a deeply sad expression, eyes watering, '
            'rain classroom door closes behind him showing rain plants still inside, '
            'empty quiet hallway stretches ahead, '
            'cactus boy looks small and alone in the empty corridor, '
            'mist still visible under the door he just left, '
            + STYLE,
    'fallback': 'Why am I so different...'
  },

  # ══════ ACT 4: MONKEY ORCHID ═══════════════════════════════════════
  {
    'chars': ['orchid'],
    'act':   'Monkey Orchid',
    'scene_context':
      'The Monkey Orchid, a plant from the mountain classroom, watches through the window '
      'and sees what happened to the cactus boy, feeling sorry for him.',
    'speaker': 'Monkey Orchid',
    'img':  MONKEY_ORC + ', '
            'peeks carefully through mountain classroom window into the hallway, '
            'watching the sad cactus boy sitting alone outside with a sympathetic concerned expression, '
            'mountain classroom behind orchid has rocky textures and cool blue-grey tones, '
            'orchid frames face with leaf hands pressed against the glass, '
            'visible contrast between mountain classroom cool interior and warm hallway outside, '
            + STYLE,
    'fallback': 'Poor cactus boy looks so sad.'
  },
  {
    'chars': ['boy', 'orchid'],
    'act':   'Monkey Orchid',
    'scene_context':
      'The Monkey Orchid flower walks out and approaches the sad cactus boy in the hallway, '
      'wanting to cheer him up.',
    'speaker': 'Monkey Orchid',
    'img':  MONK_S + ' walks carefully toward ' + BOY_S + ' in school hallway, '
            'cactus boy sits alone on a small bench against the wall looking miserable, '
            'orchid approaches with a kind warm smile and gentle open arms, '
            'empty hallway with different envoiroment classroom doors in background, '
            'soft warm lighting, orchid casts a friendly shadow toward the cactus, '
            + STYLE,
    'fallback': 'Hello! Are you alright, friend?'
  },
  {
    'chars': ['boy', 'orchid'],
    'act':   'Monkey Orchid',
    'scene_context':
      'The Monkey Orchid flower finds the cactus boy trying to pull and remove his own thorns, '
      'hurting himself, desperately wanting to be close the rain plants.',
    'speaker': 'Monkey Orchid',
    'img':  MONK_S + ' watches in alarm and shock, '
            + BOY_S + ' pulls painfully at his own spines trying to yank them out, '
            'a few removed spines lie on the hallway floor beside him, '
            'cactus boy has a desperate determined but pained expression, '
            'orchid raises leaf hands in alarm saying stop, '
            'hallway background, dramatic lighting showing the urgency of the scene, '
            + STYLE,
    'fallback': 'Stop! Do not remove your thorns!'
  },
  {
    'chars': ['boy', 'orchid'],
    'act':   'Monkey Orchid',
    'scene_context':
      'The Monkey Orchid plant gives the cactus boy wise advice: every being is created for '
      'its own environment. Changing that causes great harm.',
    'speaker': 'Monkey Orchid',
    'img':  MONK_S + ' speaks earnestly with gentle petal hand on '
            + BOY_S + ' shoulder, '
            'both face each other in school hallway, orchid has a wise kind expression, '
            'cactus boy listens with a conflicted uncertain expression, '
            'soft warm light surrounds them like a gentle spotlight, '
            'school hallway blurs softly in background, '
            'intimate moment of friendship and advice, '
            + STYLE,
    'fallback': 'Each plant belongs in its world.'
  },
  {
    'chars': ['boy', 'orchid'],
    'act':   'Monkey Orchid',
    'scene_context':
      "The cactus boy does not listen. He laughs and makes fun of the Monkey Orchid's "
      'funny-looking face, dismissing her advice completely.',
    'speaker': 'Cactus Boy',
    'img':  CACTUS_BOY + ' laughs rudely pointing finger at ' + MONK_S + ' monkey petal face, '
            'orchid flower boy looks hurt and deeply surprised, steps back with sad expression, '
            'cactus boy leans forward mockingly, eyes squinted in mean laughter, '
            'school hallway background, '
            "orchid's flower boy his friendly smile fades to sadness, "
            'cactus boy does not realize how unkind he is being, '
            + STYLE,
    'fallback': "Ha! just look at your face before you talk, it's l looks funny!"
  },

  # ══════ ACT 5: BAD DECISION ════════════════════════════════════════
  {
    'chars': ['boy'],
    'act':   'Bad Decision',
    'scene_context':
      'The cactus boy sits in the school bathroom, pulling out his remaind thorns one by one, '
      'determined to go into the rain classroom without hurting anyone.',
    'speaker': 'Cactus Boy',
    'img':  CACTUS_BOY + ', '
            'sits on floor of school bathroom in front of large mirror, '
            'pulls out own thorns one by one with a grimace of pain and determination, '
            'removed green spines pile on the tiled floor around him, '
            'bathroom has white tiles, school sink visible in background mirror reflection, '
            'determined but pained expression, focused on his goal, '
            'warm overhead bathroom lighting, '
            + STYLE,
    'fallback': 'Now I can fit in with them!'
  },
  {
    'chars': ['boy'],
    'act':   'Bad Decision',
    'scene_context':
      'Without his thorns, the cactus boy looks very different, smoother. He smiles '
      'and walks confidently toward the rain classroom door.',
    'speaker': 'Cactus Boy',
    'img':  CACTUS_BOY + ' without spines, smooth rounded green body, '
            'walks proudly with a wide confident smile toward rain classroom door, '
            'school hallway stretches ahead, rain classroom door with green glow ahead, '
            'cactus boy holds arms slightly out as if showing off his spineless body, '
            'looks satisfied and certain, unaware of the danger, '
            'bright hallway lighting, '
            + STYLE,
    'fallback': 'Now they will accept me too!'
  },
  {
    'chars': ['boy', 'rain'],
    'act':   'Bad Decision',
    'scene_context':
      'Without thorns, the cactus boy enters the rain classroom and the rain plants '
      'welcome him happily now, he feels accepted and joyful.',
    'speaker': 'Rain Plant',
    'img':  RAIN_S + ' claps and cheers welcoming '
            + BOY_S + ' without spines into the rain classroom, '
            'other tropical plants wave and smile, one offers a leaf handshake, '
            'rain classroom wet and misty as always with puddles on the floor, '
            'cactus boy beams with the biggest smile showing total happiness, '
            'water droplets begin to fall on him immediately, '
            'colorful tropical classroom atmosphere, '
            + STYLE,
    'fallback': 'Welcome! Come play with us!'
  },
  {
    'chars': ['boy', 'rain'],
    'act':   'Getting Hurt',
    'scene_context':
      'Soon the cactus boy starts feeling unwell. The wet humid environment of the rain '
      'classroom is hurting him. He looks pale and dizzy.',
    'speaker': 'Cactus Boy',
    'img':  CACTUS_BOY + ' without spines looks pale yellowish-green and dizzy, '
            'holds tiny hand to forehead feeling sick, '
            'rain classroom completely soaked with water puddles and heavy dripping mist, '
            'rain plants carry on around him not noticing, '
            'cactus body absorbing too much water, slightly swollen and discolored, '
            'distressed nauseated expression, '
            'dramatic contrast between cheerful plants and suffering cactus boy, '
            + STYLE,
    'fallback': 'I feel so sick from all this water.'
  },
  {
    'chars': ['boy'],
    'act':   'Getting Hurt',
    'scene_context':
      'The cactus boy slumps onto a desk in the rain classroom, looking very unwell '
      'and wilting from the excess water and humidity.',
    'speaker': 'Cactus Boy',
    'img':  CACTUS_BOY + ' without spines slumped over wet classroom desk, '
            'body turning yellowish from water overload, limp and exhausted, '
            'rain drips from ceiling onto him making puddles on his desk, '
            'rain plants look on with concern from their own desks, '
            'cactus clearly wilting and sick from excess water, '
            'gloomy lighting contrast with otherwise cheerful tropical classroom, '
            + STYLE,
    'fallback': 'This is hurting me so badly...'
  },

  # ══════ ACT 6: WALK HOME AND BIRD ATTACK ═══════════════════════════
  {
    'chars': ['boy'],
    'act':   'End of School',
    'scene_context':
      'The school day ends. The cactus boy, still sick and without thorns, '
      'calls his father on his phone to ask to be picked up.',
    'speaker': 'Cactus Boy',
    'img':  CACTUS_BOY + ' without spines, standing at empty school entrance after school, '
            'holds small phone to face with a weak sick expression, '
            'school building behind him, other students have all left, '
            'afternoon sun low in sky, long shadows stretch across the ground, '
            'cactus boy looks pale yellow and vulnerable, '
            + STYLE,
    'fallback': 'Father, please come get me now!'
  },
  {
    'chars': ['boy'],
    'act':   'End of School',
    'scene_context':
      'Father answers and says he is very busy and cannot come to pick up '
      'the cactus boy, who looks devastated hearing this.',
    'speaker': 'Cactus Boy',
    'img':  CACTUS_BOY + ' without spines holds phone showing busy signal on screen, '
            'stands at school gate completely alone, '
            'utterly shocked and devastated expression, eyes wide with despair, '
            'school building empty behind him, gates closing, '
            'afternoon desert light turning orange, '
            'small and vulnerable figure stranded alone, '
            + STYLE,
    'fallback': 'Father is not coming for me...'
  },
  {
    'chars': ['boy'],
    'act':   'Walk Home',
    'scene_context':
      'The cactus boy walks home alone through desert, sick from water damage '
      'and vulnerable without his protective thorns.',
    'speaker': 'Cactus Boy',
    'img':  CACTUS_BOY + ' without spines, '
            'trudges alone down a desert road under a blazing hot sun, '
            'wide empty desert landscape: sand dunes, dried rocks, no shade anywhere, '
            'cactus boy sweating and struggling, looks pale yellow and weak, '
            'hunched posture from exhaustion and sickness, '
            'harsh golden afternoon light, dramatic isolated atmosphere, '
            + STYLE,
    'fallback': 'So tired. I feel so very weak.'
  },
  {
    'chars': ['boy'],
    'act':   'Bird Attack',
    'scene_context':
      'Hungry desert birds spot the cactus boy from above. Without his thorns '
      'he looks like easy prey. They circle above him menacingly.',
    'speaker': 'Cactus Boy',
    'img':  CACTUS_BOY + ' without spines looks up in terror at the sky, '
            'three large brown-feathered desert birds circle menacingly overhead, '
            'birds cast large dark shadows over the cactus boy below, '
            'cactus boy shields his head with tiny arms, '
            'wide open desert landscape, hot afternoon sky, '
            'dramatic threatening bird silhouettes against glaring sun, '
            "fear and vulnerability evident in cactus boy's expression, "
            + STYLE,
    'fallback': 'Those birds are looking at me!'
  },
  {
    'chars': ['boy'],
    'act':   'Bird Attack',
    'scene_context':
      'The birds dive and attack the thornless cactus boy, who runs in terror '
      'across the hot desert trying to escape them.',
    'speaker': 'Cactus Boy',
    'img':  CACTUS_BOY + ' without spines sprints in absolute terror across desert, '
            'large brown birds dive aggressively from above pecking and grabbing, '
            'cactus boy runs with arms waving frantically, '
            'raised dust cloud around running feet, '
            'desert landscape with rocks and sand, hot afternoon sun, '
            'dramatic action scene with motion lines, '
            'birds outnumber and overwhelm the defenseless cactus, '
            + STYLE,
    'fallback': 'Help! They are attacking me!'
  },
  {
    'chars': ['boy'],
    'act':   'Bird Attack',
    'scene_context':
      "As the birds attack him, the cactus boy suddenly remembers his father's advice "
      "and the Orchid's flower boy warning about removing his thorns.",
    'speaker': 'Cactus Boy',
    'img':  CACTUS_BOY + ' without spines running, '
            "large glowing thought bubble above head shows father's kind wise face with glasses, "
            "and Monkey Orchid's flower boy friendly face with a warning expression, "
            'birds still dive-attacking from above and behind, '
            "cactus boy's expression changes from terror to sudden realization and regret, "
            'desert background, tears in eyes from regret, '
            + STYLE,
    'fallback': 'Father was right about everything!'
  },

  # ══════ ACT 7: RESCUE AND RESOLUTION ═══════════════════════════════
  {
    'chars': ['boy', 'dad'],
    'act':   'Rescue',
    'scene_context':
      'Father cactus, driving home from work through the desert, spots his son '
      'being attacked by birds and reacts in shock.',
    'speaker': 'Father Cactus',
    'img':  DAD_S + ' drives green car on desert road, '
            'eyes go wide in absolute shock through the windshield, '
            'through windshield ahead: ' + BOY_S + ' running with birds attacking him, '
            "father's hands grip steering wheel tightly in emergency reaction, "
            'foot on brake, dust rising under tires, '
            'desert road scene with urgent dramatic lighting, '
            + STYLE,
    'fallback': 'My son! Hold on, I am coming!'
  },
  {
    'chars': ['boy', 'dad'],
    'act':   'Rescue',
    'scene_context':
      'Father speeds the car toward his son and opens the car door, shouting for '
      'the cactus boy to jump in quickly while birds circle.',
    'speaker': 'Father Cactus',
    'img':  DAD_S + ' swings car door wide open urgently from driver seat, '
            + BOY_S + ' sprints desperately toward the open car door arms outstretched, '
            'brown birds still diving aggressively from above, '
            'car stopped in dusty desert road with tire skid marks, '
            'father leans out shouting and waving son in, '
            + STYLE,
    'fallback': 'Get in the car quickly, son!'
  },
  {
    'chars': ['boy', 'dad'],
    'act':   'Rescue',
    'scene_context':
      'The cactus boy jumps safely inside the car. Father drives away fast. '
      'The birds flap away as the car speeds off into the desert.',
    'speaker': 'Cactus Boy',
    'img':  BOY_S + ' safely buckled in passenger seat car door slamming shut, '
            + DAD_S + ' accelerates away fast on desert road, '
            'birds recede in the rear window getting smaller, '
            'cactus boy breathes deeply with huge relief, '
            'father drives with determined protective expression, '
            'golden desert afternoon light, safe escape, '
            + STYLE,
    'fallback': 'Thank you Father! I am safe now.'
  },
  {
    'chars': ['boy', 'dad'],
    'act':   'Driving Home',
    'scene_context':
      'Father cactus drives calmly now, looking at his sick thornless son with concern '
      'and care, asking gently what happened today.',
    'speaker': 'Father Cactus',
    'img':  DAD_S + ' drives home quietly, '
            'glances with deep concern and love at ' + BOY_S + ' in passenger seat, '
            'cactus boy looks pale yellow and exhausted without spines, '
            'desert landscape at golden sunset through car windows, '
            'father has a worried but gentle patient expression, '
            'warm soft sunset light fills the car with orange glow, '
            + STYLE,
    'fallback': 'Tell me what happened!'
  },

  # ══════ ACT 8: LESSON AND RESOLUTION ═══════════════════════════════
  {
    'chars': ['boy', 'dad'],
    'act':   'Resolution',
    'scene_context':
      'At home, the cactus boy tells his father everything that happened at school, '
      'crying and feeling regretful about all his bad choices.',
    'speaker': 'Cactus Boy',
    'img':  BOY_S + ' sits with ' + DAD_S + ' together on a cozy sofa at home, '
            'cactus boy cries with tears rolling down cheeks, '
            'gestures explaining everything with tiny arms, '
            'father sits closely listening with patient loving expression, '
            'cozy desert home living room with warm lamp light, '
            'cactus paintings and family photos on sandy walls, '
            'intimate emotional family scene, '
            + STYLE,
    'fallback': 'I am so sorry I did not listen.'
  },
  {
    'chars': ['boy', 'dad'],
    'act':   'Resolution',
    'scene_context':
      'The cactus boy says to his father: I should have listened to you and '
      'followed your advice. He regrets everything deeply.',
    'speaker': 'Cactus Boy',
    'img':  BOY_S + ' looks directly at ' + DAD_S + ' with deep regret and sincerity, '
            "grabs father's arm with both tiny hands earnestly, "
            'eyes red from crying, expression full of genuine remorse, '
            'father watches his son with soft forgiving eyes, '
            'warm home interior, gentle lamp light, '
            'heartfelt apology moment, '
            + STYLE,
    'fallback': 'I should have listened to you!'
  },
  {
    'chars': ['boy', 'dad'],
    'act':   'Resolution',
    'scene_context':
      'Father cactus smiles warmly, hugs his son, and says: Everything is fine '
      'as long as you are safe, and you have learned from your mistakes.',
    'speaker': 'Father Cactus',
    'img':  DAD_S + ' wraps both arms around ' + BOY_S + ' in a warm gentle hug, '
            'father smiles with pure relief and love, '
            'cactus boy hugs back with eyes closed peacefully, '
            'cozy warm home interior, golden lamp light on both of them, '
            'potted cactus plants visible in background, '
            'beautiful tender father-son moment of forgiveness and love, '
            + STYLE,
    'fallback': 'I am glad you are safe, my son.'
  },
  {
    'chars': ['boy', 'dad'],
    'act':   'Resolution',
    'scene_context':
      'Father tells his son: the good thing is that you have learned from your mistakes '
      'and now you know what is right. Son nods and smiles.',
    'speaker': 'Father Cactus',
    'img':  DAD_S + ' holds ' + BOY_S + " at arm's length gently, "
            'speaks with a warm wise smile, '
            'cactus boy nods slowly with a small peaceful smile of understanding, '
            'eyes still slightly wet but now calm and wise, '
            'home background with warm evening light, '
            'both look at each other with mutual love and understanding, '
            + STYLE,
    'fallback': 'Learning from mistakes is wisdom!'
  },
  {
    'chars': ['boy'],
    'act':   'Happy Ending',
    'scene_context':
      'Next morning, the cactus boy wakes up happy, his thorns are growing back, '
      'and he looks at himself in the mirror proud of who he is.',
    'speaker': 'Cactus Boy',
    'img':  CACTUS_BOY + ', '
            'stands in front of large bedroom mirror smiling with pure pride and happiness, '
            'new small green spines visibly growing back all over his body, '
            'points at his own reflection with joy, '
            'bright sunny desert morning bedroom, golden sunlight from window, '
            'red school backpack ready on the floor behind him, '
            'confident joyful expression, completely at peace with who he is, '
            + STYLE,
    'fallback': 'I am proud to be a cactus!'
  },
]

# aliases
scenes        = STORY_OUTLINE
image_prompts = [s['img'] for s in STORY_OUTLINE]

print(f'✅ Story outline: {len(STORY_OUTLINE)} scenes across 8 acts')
print()
# Token count report
from collections import Counter
acts = Counter(s['act'] for s in STORY_OUTLINE)
tcs = []
print(f'  {"Act":<22} {"Scenes":<8} {"Avg tok":<10}')
print('  ' + '─'*42)
for act, cnt in acts.items():
    act_scenes = [s for s in STORY_OUTLINE if s['act']==act]
    avg = sum(token_count(s['img']) for s in act_scenes) / len(act_scenes)
    tcs.extend([token_count(s['img']) for s in act_scenes])
    print(f'  {act:<22} {cnt:<8} {avg:<10.0f}')
print()
print(f'  Total scenes     : {len(STORY_OUTLINE)}')
print(f'  Max prompt tokens: {max(tcs)} (FLUX T5 limit: 512 ✅)')
print(f'  Avg prompt tokens: {sum(tcs)/len(tcs):.0f}')
over = [(i+1, token_count(s["img"])) for i,s in enumerate(STORY_OUTLINE) if token_count(s["img"])>512]
if over:
    print(f'  ⚠️  Over 512: {over}')
else:
    print(f'  ✅ All prompts within FLUX T5 512-token limit')


# 🦙 STEP 6 — LLAMA 3.1 8B VIA GROQ API + LANGCHAIN

**This is where Llama 3.1 8B Instruct is used — via Groq API.**

For each of the 41 scenes, a structured prompt is sent to Groq's hosted Llama 3.1 8B.
Llama generates a warm, character-appropriate dialogue line (10–25 words).

Why Groq API + LangChain:
- **No HuggingFace gate** — works immediately with just an API key
- **Zero VRAM** — inference runs on Groq's LPU hardware remotely
- **<1 second per scene** — faster than loading a local model
- **Same model quality** as running Llama 3.1 8B locally
- LangChain handles retries, structured chat messages, and clean output


In [ ]:
print('🦙 Initializing Llama 3.1 8B via Groq API + LangChain...')

groq_llm = ChatGroq(
    model       = 'llama-3.1-8b-instant',   # Groq-hosted Llama 3.1 8B
    api_key     = GROQ_API_KEY,
    temperature = 0.65,
    max_tokens  = 100,
)

# Quick connectivity test
test_resp = groq_llm.invoke([HumanMessage(content='Say: ready')])
print(f'✅ Groq API connected — Llama 3.1 8B ready')
print(f'   Test response: {test_resp.content.strip()[:50]}')
print(f'   VRAM used    : 0 GB (runs remotely on Groq LPU)')


# 🦙 STEP 6B — COMPOSE ALL 41 DIALOGUES WITH LLAMA 3.1 8B (GROQ)


In [ ]:
CHAR_VOICE = {
    'Cactus Boy': (
        'a young curious cactus boy, enthusiastic and emotional, '
        'speaks in short excited or worried sentences like a child aged 7-8'
    ),
    'Father Cactus': (
        'a wise kind cactus father, speaks warmly and patiently, '
        'gives gentle life advice, calm reassuring tone'
    ),
    'Monkey Orchid': (
        'a friendly helpful orchid plant boy with a monkey-face flower, '
        'speaks with care and concern, kind and wise but gentle'
    ),
    'Rain Plant': (
        'a tropical rain plant student, speaks with surprise or laughter, '
        'reactions are exaggerated like a child'
    ),
}

def compose_dialogue(scene_context, speaker, fallback, max_words=25):
    """
    Calls Llama 3.1 8B via Groq API + LangChain to compose
    a natural dialogue line for one comic panel.
    """
    voice = CHAR_VOICE.get(speaker, 'a plant character in a children\'s story')

    system_msg = SystemMessage(content=(
        'You are writing speech bubble dialogue for a children\'s comic book. '
        'The story is about a small cactus boy who learns to be proud of who he is. '
        'RULES: Write ONLY the spoken words (10 to 25 words). '
        'NO quotes, NO character name, NO stage directions, NO explanation. '
        'Make it emotional, vivid, and true to the scene. '
        'Tone: warm and simple for children aged 4-12.'
    ))

    user_msg = HumanMessage(content=(
        f'EXAMPLES of good dialogue lines:\n'
        f'  Scene: Boy wakes up happy on first school day\n'
        f'  Cactus Boy: I cannot wait to meet everyone at school today\n'
        f'\n'
        f'  Scene: Father gives advice about accepting yourself\n'
        f'  Father Cactus: Every plant was made perfectly for its own home\n'
        f'\n'
        f'  Scene: Plants laugh at cactus boy because of his spines\n'
        f'  Rain Plant: Your sharp thorns will hurt us so please stay away\n'
        f'\n'
        f'CHARACTER: {speaker} — {voice}\n'
        f'SCENE: {scene_context}\n'
        f'\n'
        f'Write {speaker}\'s one spoken line (10-25 words):'
    ))

    try:
        response = groq_llm.invoke([system_msg, user_msg])
        raw = response.content.strip()

        # Clean Groq/LangChain output
        raw = re.sub(r'(?i)^(here is|here\'s|output:|dialogue:|says?:|response:)[^:]*:\s*', '', raw).strip()
        raw = re.sub(r'(?i)^[A-Za-z ]+says?:\s*', '', raw).strip()
        raw = re.sub(r'["\*#<>_]', '', raw).strip()

        # Take first sentence
        sentence = re.split(r'(?<=[.!?])\s', raw)[0].strip()
        sentence = re.sub(r'[.!?]+$', '', sentence).strip()
        words = sentence.split()

        if 5 <= len(words) <= max_words:
            return words[0].capitalize() + ' ' + ' '.join(words[1:])
        if len(words) > max_words:
            w = words[:max_words]
            return w[0].capitalize() + ' ' + ' '.join(w[1:])

        # Try full raw line
        line_words = raw.split('\n')[0].strip().split()
        if 5 <= len(line_words) <= max_words:
            return line_words[0].capitalize() + ' ' + ' '.join(line_words[1:])

        return fallback

    except Exception as e:
        print(f'  ⚠️  Groq API error scene {i}: {e} — using fallback')
        return fallback


# ── Compose all 41 dialogues ──────────────────────────────────────
import time
print('🗣️  Composing dialogues with Llama 3.1 8B via Groq...\n')
print(f'{"#":<5} {"Speaker":<16} {"Dialogue":<46} {"Tok":<5} {"Src"}')
print('─' * 82)

groq_dialogues = []
for i, scene in enumerate(STORY_OUTLINE, 1):
    dialogue = compose_dialogue(
        scene['scene_context'],
        scene['speaker'],
        scene['fallback']
    )
    source = 'Groq' if dialogue != scene['fallback'] else 'fallback'
    tok = token_count(dialogue)
    groq_dialogues.append(dialogue)
    print(f'{i:<5} {scene["speaker"]:<16} {dialogue:<46} {tok:<5} {source}')
    time.sleep(0.5)   # Groq free tier: ~30 req/min → 0.5s gap is safe

# Aliases so rest of notebook works unchanged
flan_dialogues  = groq_dialogues
llama_dialogues = groq_dialogues
gemma_dialogues = groq_dialogues

n_groq     = sum(1 for i,s in enumerate(STORY_OUTLINE) if groq_dialogues[i] != s['fallback'])
n_fallback = len(groq_dialogues) - n_groq
print(f'\n✅ {len(groq_dialogues)} dialogues composed via Groq API')
print(f'   Llama-generated : {n_groq}')
print(f'   Fallback used   : {n_fallback}')


# ✅ STEP 6C — NO FREE RAM NEEDED (Groq runs remotely)

Llama 3.1 8B ran on Groq's LPU servers — zero local VRAM was used.
No cleanup needed. All 41 dialogues are stored in `groq_dialogues`.
Ready to load SD 3.5 Large.


In [ ]:
# ── Build final SCENES list from outline + Groq/Llama dialogues ──
SCENES = []
for outline, dialogue in zip(STORY_OUTLINE, groq_dialogues):
    SCENES.append({
        'chars':    outline['chars'],
        'act':      outline['act'],
        'img':      outline['img'],
        'dialogue': dialogue,
    })

scenes        = SCENES
image_prompts = [s['img'] for s in SCENES]

print(f'✅ SCENES list built: {len(SCENES)} scenes')
print('   Each scene: chars, act, img (→ FLUX), dialogue (from Llama/Groq)')


# 🎨 STEP 7 — LOAD FLUX.1-dev

**FLUX.2-dev 4-bit quantized (diffusers/FLUX.1-dev):**
- T5-XXL text encoder (256 tokens) — full prompt processed
- 30 inference steps recommended for this version
- guidance_scale = 3.5

**CRITICAL for style:**
FLUX does **not** use negative_prompt (architecture ignores it completely).
ALL style control comes from the positive STYLE string which explicitly repeats:
"hand-drawn", "flat", "2D cartoon", "NOT realistic", "NOT 3D".

Model: `diffusers/FLUX.1-dev`  |  VRAM: ~33 GB


In [ ]:
from diffusers import FluxPipeline
from transformers import CLIPTextModel, CLIPTokenizer
import torch

# 1. Load the Long-CLIP components (248 token limit)
long_clip_model_id = "zer0int/LongCLIP-GmP-ViT-L-14"

tokenizer = CLIPTokenizer.from_pretrained(long_clip_model_id)
text_encoder = CLIPTextModel.from_pretrained(long_clip_model_id, torch_dtype=torch.bfloat16)

print('🎨 Loading standard FLUX.1-dev (bfloat16)...')

# This avoids the 'meta tensor' errors associated with the 4-bit experimental repo.
flux_pipe = FluxPipeline.from_pretrained(
    'black-forest-labs/FLUX.1-dev',
    tokenizer=tokenizer,
    text_encoder=text_encoder,
    torch_dtype=torch.bfloat16
).to(device)

print('✅ FLUX.1-dev loaded successfully')
if torch.cuda.is_available():
    used = torch.cuda.memory_allocated() / 1024**3
    total = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f'   VRAM used : {used:.1f} GB / {total:.1f} GB')

print('\nChecking prompt lengths...')
tcs = [token_count(s["img"]) for s in SCENES]
print(f'  Max : {max(tcs)} tokens  |  Avg: {sum(tcs)/len(tcs):.1f}')
print(f'  T5-XXL limit: 256 tokens ✅')

# 🖼️ STEP 8 — GENERATE 41 IMAGES WITH FLUX.1-dev

**FLUX.1-dev settings:**
- `num_inference_steps = 30`  
- `guidance_scale = 3.5`      —
- `negative_prompt` = NOT USED (FLUX ignores it — kept for documentation only)
- Resolution: `1024×1024`

**How Donald Duck 2D style is enforced:**
Since FLUX ignores negative_prompt, the STYLE string contains explicit positive terms:
- "hand-drawn 2D flat cartoon", "vintage disney comic style"
- "NOT realistic, NOT 3D rendered, NOT photographic"
- "completely flat bright colors with absolutely no shading or gradients"

This guides FLUX's T5-XXL encoder toward pure flat cartoon and away from realism/3D.


In [ ]:
print(f'🎨 Generating {len(SCENES)} panels with FLUX.1-dev at 1024×1024...\n')

images = []

for i, scene in enumerate(SCENES, 1):
    char_names = '+'.join(CHARS[c]['name'] for c in scene['chars'] if c in CHARS)
    tc = token_count(scene['img'])
    act = scene['act'][:18]
    print(f'[{i:2d}/{len(SCENES)}] {char_names:<18} | {tc:>3} tok | [{act}] "{scene["dialogue"][:32]}"')

    seed = sum(CHARS[c]['seed'] for c in scene['chars'] if c in CHARS)
    generator = torch.Generator(device='cpu').manual_seed(seed)

    with torch.no_grad():
        image = flux_pipe(
            prompt              = scene['img'],
            # negative_prompt not passed — FLUX ignores it anyway
            max_sequence_length = 256,
            num_inference_steps = 30,
            guidance_scale      = 3.5,
            generator           = generator,
            height              = 1024,
            width               = 1024,
        ).images[0]

    path = f'comic/images/scene_{i:03d}.png'
    image.save(path)
    images.append(path)

    if i % 1 == 0 or i == len(SCENES):
        display(image.resize((300, 300)))

    gc.collect()
    torch.cuda.empty_cache()

print(f'\n✅ All {len(images)} panels generated with FLUX.1-dev!')


# 🗑️ FREE RAM B — After FLUX.1-dev


In [ ]:
print('🗑️  Freeing FLUX.1-dev')
del flux_pipe
gc.collect()
torch.cuda.empty_cache()
if device == 'cuda':
    free = (torch.cuda.get_device_properties(0).total_memory
            - torch.cuda.memory_allocated()) / 1024**3
    print(f'   VRAM free: {free:.1f} GB')
print('✅ FLUX.1-dev freed')


# 💬 STEP 9 — ADD SPEECH BUBBLES (Llama 3.1 8B / Groq dialogue)

Font size: **50px bold** on 1024×1024 images.
Dynamic bubble height — grows to fit text automatically.
Dialogue in each bubble was composed by Llama 3.1 8B via Groq API.


In [ ]:
def add_bubble(img_path, text, out_path):
    img  = Image.open(img_path).convert('RGB')
    draw = ImageDraw.Draw(img)
    W, H = img.size   # 1024 × 1024

    # ── Font 50px bold ────────────────────────────────────────────
    FONT_SIZE = 50
    font = None
    for fp in [
        '/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf',
        '/usr/share/fonts/truetype/liberation/LiberationSans-Bold.ttf',
        '/usr/share/fonts/truetype/freefont/FreeSansBold.ttf',
    ]:
        try: font = ImageFont.truetype(fp, FONT_SIZE); break
        except: continue
    if font is None: font = ImageFont.load_default()

    # ── Bubble geometry ───────────────────────────────────────────
    MARGIN = 40; PAD = 20
    bx = MARGIN; bw = W - 2*MARGIN
    LINE_H = FONT_SIZE + 12

    # Word-wrap at 50px
    words = text.split(); lines, cur = [], ''
    for w in words:
        test = (cur+' '+w).strip()
        if draw.textbbox((0,0), test, font=font)[2] < bw - 2*PAD: cur = test
        else:
            if cur: lines.append(cur)
            cur = w
    if cur: lines.append(cur)

    bh = len(lines)*LINE_H + 2*PAD
    by = 25

    # ── Draw white rounded bubble ─────────────────────────────────
    draw.rounded_rectangle([(bx,by),(bx+bw,by+bh)],
                           radius=24, fill='white', outline='black', width=6)
    mid = bx + bw//2
    draw.polygon([(mid-20,by+bh),(mid+20,by+bh),(mid,by+bh+36)],
                 fill='white', outline='black')

    # ── Text ──────────────────────────────────────────────────────
    ty = by + PAD
    for line in lines:
        bb = draw.textbbox((0,0), line, font=font)
        tx = bx + (bw-(bb[2]-bb[0]))//2
        draw.text((tx,ty), line, fill='black', font=font)
        ty += LINE_H
    img.save(out_path)


images_final = []
for i, (path, scene) in enumerate(zip(images, SCENES), 1):
    out = f'comic/images/scene_{i:03d}_final.png'
    add_bubble(path, scene['dialogue'], out)
    images_final.append(out)

images = images_final
print(f'✅ Speech bubbles (50px bold, FLAN dialogue) — all {len(images)} panels done')

# Preview
preview = Image.open(images[0]).crop((0, 0, 1024, 260))
display(preview.resize((600, 152)))
print('↑ Preview — text should be large and clearly readable')


# 📊 STEP 10 — LOAD LongCLIP (evaluation only)



In [ ]:
!pip install git+https://github.com/openai/CLIP.git

In [ ]:
import clip

clip_tok_check = CLIPTokenizer.from_pretrained('openai/clip-vit-large-patch14')
def token_count(text):
    return len(clip_tok_check.encode(text, add_special_tokens=False))

clip_model, clip_preprocess = clip.load('ViT-L/14', device=device)
clip_model.eval()
clip_tokenizer = clip.tokenize
print('✅ CLIP loaded')
print(f'   Image dim: {clip_model.visual.output_dim}')


# 📊 STEP 10B — LONGCLIP EVALUATION


In [ ]:
def calculate_clip_score(image_path, clip_text):
    """
    Calculate CLIP score using a short clip_text (≤256 tokens).
    clip_text should be scene_context, not the full image_prompt.
    """
    image       = Image.open(image_path).convert('RGB')
    image_input = clip_preprocess(image).unsqueeze(0).to(device)
    # Warn if still over 77 tokens
    tc = token_count(clip_text)
    if tc > 77:
        print(f'  ⚠️  clip_text {tc} tokens — truncating to 256')
    text_input  = clip_tokenizer([clip_text], truncate=True).to(device)
    with torch.no_grad():
        image_features = clip_model.encode_image(image_input)
        text_features  = clip_model.encode_text(text_input)
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)
        text_features  = text_features  / text_features.norm(dim=-1, keepdim=True)
        similarity     = (image_features @ text_features.T).item()
    return similarity * 100

# ── Use scene_context for CLIP (≤256 tok) not full image_prompts ───
clip_eval_texts = [s['scene_context'] for s in STORY_OUTLINE]

# Verify all are within 256 tokens
max_tc = max(token_count(t) for t in clip_eval_texts)
print(f'  Max scene_context tokens: {max_tc} (limit: 256) {"✅" if max_tc<=256 else "⚠️"}')
print()

original_images = [f'comic/images/scene_{i:03d}.png' for i in range(1, len(SCENES)+1)]

clip_scores = []
print('='*72)
print('CLIP SCORE RESULTS  (image vs scene_context — accurate ≤256 token text)')
print('='*72)
print(f'{"Sc":<5} {"CLIP":<9} {"Rating":<14} {"Act":<18} {"Scene Context (first 40 chars)":<40}')
print('-'*72)

for i in range(len(scenes)):
    score = calculate_clip_score(original_images[i], clip_eval_texts[i])
    clip_scores.append(score)
    if   score >= 30: rating = '⭐ Excellent'
    elif score >= 25: rating = '✅ Good'
    elif score >= 20: rating = '🟡 Fair'
    else:             rating = '🔴 Low'
    act     = scenes[i]['act'][:16]
    ctx_preview = clip_eval_texts[i][:38]
    print(f'{i+1:<5} {score:<9.2f} {rating:<14} {act:<18} {ctx_preview}')

avg_clip = float(np.mean(clip_scores))
print('-'*72)
print(f'{"AVG":<5} {avg_clip:<9.2f}')
print('='*72)
print(f'\n📊 Summary:  Avg={avg_clip:.2f}  Min={min(clip_scores):.2f}  Max={max(clip_scores):.2f}')
print(f'   Excellent(≥30): {sum(1 for s in clip_scores if s>=30)}  '
      f'Good(25-30): {sum(1 for s in clip_scores if 25<=s<30)}  '
      f'Fair(20-25): {sum(1 for s in clip_scores if 20<=s<25)}  '
      f'Low(<20): {sum(1 for s in clip_scores if s<20)}')
print()
print('  Note: These scores evaluate scene concept alignment.')
print('  FLUX image generation was driven by the full 150-250 token prompts via T5-XXL.')


# 📝 STEP 11 — PERPLEXITY (story coherence)

In [ ]:
print('📝 Computing perplexity on Llama/Groq-generated dialogue...')
perp_tok = GPT2TokenizerFast.from_pretrained('gpt2')
perp_mdl = GPT2LMHeadModel.from_pretrained('gpt2').to(device)
perp_tok.pad_token = perp_tok.eos_token
dialogue_text = ' '.join(s['dialogue'] for s in SCENES)
enc = perp_tok(dialogue_text, return_tensors='pt', truncation=True, max_length=1024).to(device)
with torch.no_grad():
    perplexity = torch.exp(perp_mdl(enc['input_ids'], labels=enc['input_ids']).loss).item()
print(f'Perplexity: {perplexity:.2f}  (target < 50)  {"✅" if perplexity < 50 else "⚠️"}')
print(f'Dialogue sample: "{dialogue_text[:120]}..."')


# 🗑️ FREE RAM D — After perplexity

In [ ]:
print('🗑️  Freeing GPT-2...')
del perp_mdl, perp_tok
gc.collect(); torch.cuda.empty_cache()
print('✅ RAM freed')


# 👁️ HUMAN EVALUATION 1

Review all scenes and fill in your ratings in the cell below.
Rating scale: **1=Poor · 2=Fair · 3=Good · 4=Very Good · 5=Excellent**

After filling in all evaluation parts, the data will be collected and included in the final PDF report.


In [ ]:
print('\n👁️  HUMAN EVALUATION PART 1\n')
print('After reviewing all scenes, fill in these ratings (1-5):\n')

human_eval_1 = {
    'scenes_range': '1',
    'story_coherence': {
        'score': 4,    # 1-5: Does the story flow logically in these scenes?
    },
    'visual_quality': {
        'score': 3,    # 1-5: Overall image quality and consistency
    },
    'character_consistency': {
        'score': 2,    # 1-5: Do characters look the same across these scenes?
    },
    'dialogue_quality': {
        'score': 4,    # 1-5: Is dialogue natural and appropriate for children?
    },
    'scene_composition': {
        'score': 3,    # 1-5: Clear composition and framing?
    },
    'background_detail': {
        'score': 4,    # 1-5: Backgrounds fully illustrated and detailed?
    },
    'artifacts_count': {
        'count': 6,    # How many scenes have visual artifacts/errors?
    },
    'over_all_rating': {
        'score': 4,    # 1-5: Overall quality of this section
    }
}

print('Rating scale: 1=Poor, 2=Fair, 3=Good, 4=Very Good, 5=Excellent\n')
os.makedirs('comic/evaluation', exist_ok=True)
with open('comic/evaluation/human_eval_1.json', 'w') as f:
    json.dump(human_eval_1, f, indent=2)
print(f'✅ Evaluation 1 template saved')

# 👁️ HUMAN EVALUATION 2

Review all scenes and fill in your ratings in the cell below.
Rating scale: **1=Poor · 2=Fair · 3=Good · 4=Very Good · 5=Excellent**

After filling in all evaluation, the data will be collected and included in the final PDF report.


In [ ]:
print('\n👁️  HUMAN EVALUATION PART 1\n')
print('After reviewing all scenes, fill in these ratings (1-5):\n')

human_eval_2 = {
    'scenes_range': '2',
    'story_coherence': {
        'score': 4,    # 1-5: Does the story flow logically in these scenes?
    },
    'visual_quality': {
        'score': 2,    # 1-5: Overall image quality and consistency
    },
    'character_consistency': {
        'score': 1,    # 1-5: Do characters look the same across these scenes?
    },
    'dialogue_quality': {
        'score': 2,    # 1-5: Is dialogue natural and appropriate for children?
    },
    'scene_composition': {
        'score': 2,    # 1-5: Clear composition and framing?
    },
    'background_detail': {
        'score': 3,    # 1-5: Backgrounds fully illustrated and detailed?
    },
    'artifacts_count': {
        'count': 3,    # How many scenes have visual artifacts/errors?
    },
    'over_all_rating': {
        'score': 2,    # 1-5: Overall quality of this section
    }
}

print('Rating scale: 1=Poor, 2=Fair, 3=Good, 4=Very Good, 5=Excellent\n')
os.makedirs('comic/evaluation', exist_ok=True)
with open('comic/evaluation/human_eval_2.json', 'w') as f:
    json.dump(human_eval_2, f, indent=2)
print(f'✅ Evaluation 2 template saved')

# 👁️ HUMAN EVALUATION 3

Review all scenes and fill in your ratings in the cell below.
Rating scale: **1=Poor · 2=Fair · 3=Good · 4=Very Good · 5=Excellent**

After filling in all  evaluation parts, the data will be collected and included in the final PDF report.


In [ ]:
print('\n👁️  HUMAN EVALUATION PART 1\n')
print('After reviewing all scenes, fill in these ratings (1-5):\n')

human_eval_3 = {
    'scenes_range': '3',
    'story_coherence': {
        'score': 3,    # 1-5: Does the story flow logically in these scenes?
    },
    'visual_quality': {
        'score': 4,    # 1-5: Overall image quality and consistency
    },
    'character_consistency': {
        'score': 2,    # 1-5: Do characters look the same across these scenes?
    },
    'dialogue_quality': {
        'score': 4,    # 1-5: Is dialogue natural and appropriate for children?
    },
    'scene_composition': {
        'score': 3,    # 1-5: Clear composition and framing?
    },
    'background_detail': {
        'score': 4,    # 1-5: Backgrounds fully illustrated and detailed?
    },
    'artifacts_count': {
        'count': 5,    # How many scenes have visual artifacts/errors?
    },
    'over_all_rating': {
        'score': 3,    # 1-5: Overall quality of this section
    }
}

print('Rating scale: 1=Poor, 2=Fair, 3=Good, 4=Very Good, 5=Excellent\n')
os.makedirs('comic/evaluation', exist_ok=True)
with open('comic/evaluation/human_eval_3.json', 'w') as f:
    json.dump(human_eval_3, f, indent=2)
print(f'✅ Evaluation 3 template saved')

In [ ]:
# Run after filling in ALL 3 human evaluation above (1, 2, and 3)
groq_generated = sum(1 for i,s in enumerate(STORY_OUTLINE) if groq_dialogues[i] != s['fallback'])
groq_total     = len(STORY_OUTLINE)

eval_data = {
    'story':          'The Misguided Enthusiasm of the Little Cactus Boy',
    'story_author':   'Original story by user',
    'model_image':    'FLUX.2-dev bfloat16 1024x1024 (diffusers/FLUX.1-dev)',
    'model_dialogue': 'Llama 3.1 8B Instruct via Groq API + LangChain (llama-3.1-8b-instant)',
    'model_clip':     'LongCLIP zer0int/LongCLIP-GmP-ViT-L-14 (248-token limit)',
    'scenes_count':   len(SCENES),
    'llama_dialogue': {
        'generated':  groq_generated,
        'fallback':   groq_total - groq_generated,
        'total':      groq_total,
        'dialogues':  {f's{i+1}': d for i,d in enumerate(groq_dialogues)},
    },
    'clip': {
        'model':     'LongCLIP zer0int/LongCLIP-GmP-ViT-L-14',
        'avg':       round(avg_clip, 2),
        'min':       round(float(min(clip_scores)), 2),
        'max':       round(float(max(clip_scores)), 2),
        'excellent': sum(1 for s in clip_scores if s>=30),
        'good':      sum(1 for s in clip_scores if 25<=s<30),
        'per_scene': {f's{i+1}': round(float(v),2) for i,v in enumerate(clip_scores)},
    },
    'perplexity':     round(float(perplexity), 2),
    'human_eval_1': human_eval_1,
    'human_eval_2': human_eval_2,
    'human_eval_3': human_eval_3,
}

with open('comic/evaluation/results.json', 'w') as f:
    json.dump(eval_data, f, indent=2)

print('\n' + '='*75)
print('📋 HUMAN EVALUATION SUMMARY — 3 PARTS')
print('='*75)

for part_num, part_name, eval_dict in [
    (1, 'human_eval_1', human_eval_1),
    (2, 'human_eval_2', human_eval_2),
    (3, 'human_eval_3', human_eval_3)
]:
    print(f'\n{part_name}')
    print('-'*75)
    for key, val in eval_dict.items():
        if key == 'scenes_range':
            continue
        elif key == 'artifacts_count':
            count = val.get('count', '?')
            print(f'  {"Artifacts":<28}: count={count}')
        else:
            sc    = val.get('score')
            stars = ('★'*(sc or 0) + '☆'*(5-(sc or 0))) if sc else '─────'
            notes = val.get('notes', '')[:40]
            display_name = key.replace('_', ' ').title()
            print(f'  {display_name:<28}: {stars}  {notes}')

print()
print('='*75)
print(f'Llama/Groq Dialogues: {groq_generated}/{groq_total} AI-generated, '
      f'{groq_total-groq_generated} fallback')
print('='*75)
print('✅ All evaluations saved to comic/evaluation/results.json')


# 📚 STEP 13 — COMIC BOOK PDF

In [ ]:
def make_comic_pdf(scenes, images, out='comic/The Misguided Enthusiasm of the Little Cactus Boy.pdf'):
    c = pdf_canvas.Canvas(out, pagesize=letter)
    W, H = letter

    # Cover
    c.setFillColor(colors.HexColor('#F4A836')); c.rect(0,0,W,H,fill=1)
    c.setFillColor(colors.HexColor('#2D6A4F')); c.setFont('Helvetica-Bold',44)
    c.drawCentredString(W/2, H-130, 'LITTLE CACTUS')
    c.drawCentredString(W/2, H-185, 'GOES TO SCHOOL')
    c.setFillColor(colors.HexColor('#1B4332')); c.setFont('Helvetica',15)
    c.drawCentredString(W/2, H-230, 'A story about being proud of who you are')
    c.setFont('Helvetica',10)
    c.drawCentredString(W/2, 52, f'{len(scenes)} Scenes  •  Llama 3.1 8B (Groq)  •  FLUX.1-dev 1024×1024')
    c.showPage()

    # 6 panels per page
    CW, CH = (W-50)/2, (H-55)/3
    for ps in range(0, len(scenes), 6):
        c.setFillColor(colors.HexColor('#FFFBF0')); c.rect(0,0,W,H,fill=1)
        c.setFillColor(colors.gray); c.setFont('Helvetica',9)
        c.drawCentredString(W/2, 14, str(ps//6+2))
        for slot in range(6):
            idx = ps+slot
            if idx >= len(scenes): break
            row,col = slot//2, slot%2
            x = 25+col*CW; y = H-28-(row+1)*CH; m=6
            c.setFillColor(colors.white)
            c.roundRect(x+m,y+m,CW-2*m,CH-2*m,7,fill=1,stroke=0)
            c.setStrokeColor(colors.HexColor('#2D6A4F')); c.setLineWidth(2.5)
            c.roundRect(x+m,y+m,CW-2*m,CH-2*m,7,fill=0,stroke=1)
            bx2,by2 = x+m+14,y+m+CH-2*m-14
            c.setFillColor(colors.HexColor('#2D6A4F'))
            c.circle(bx2,by2,12,fill=1,stroke=0)
            c.setFillColor(colors.white); c.setFont('Helvetica-Bold',9)
            c.drawCentredString(bx2,by2-3,str(idx+1))
            c.drawImage(images[idx], x+m+7,y+m+7,
                        width=CW-2*m-14, height=CH-2*m-14,
                        preserveAspectRatio=True, mask='auto')
        c.showPage()

    # Back cover
    c.setFillColor(colors.HexColor('#F4A836')); c.rect(0,0,W,H,fill=1)
    c.setFillColor(colors.HexColor('#2D6A4F')); c.setFont('Helvetica-Bold',52)
    c.drawCentredString(W/2,H/2+40,'THE END')
    c.setFillColor(colors.HexColor('#1B4332')); c.setFont('Helvetica',12)
    c.drawCentredString(W/2,H/2-50,' Be content with what God has allotted you, for contentment brings peace and tranquility to the heart,')
    c.drawCentredString(W/2,H/2-65,' and makes you the richest of people in this world and the hereafter.')
    c.drawCentredString(W/2,H/2-80,' Indeed, some blessings are only truly appreciated when they are gone.')
    c.drawCentredString(W/2,H/2-95,' Choose your companions carefully, for they will either lead you astray or guide you to the right path.')
    c.drawCentredString(W/2,H/2-110,'  Praise be to Allah in all circumstances')
    c.showPage(); c.save()
    return out

comic_pdf = make_comic_pdf(SCENES, images)
print(f'✅ Comic PDF: {comic_pdf}')


# 📄 STEP 14 — EVALUATION REPORT PDF

In [ ]:
def make_report(d, out='comic/Evaluation_Report.pdf'):
    doc = SimpleDocTemplate(out, pagesize=letter)
    els, sty = [], getSampleStyleSheet()

    # === TITLE AND METHODOLOGY ===
    els += [
        Paragraph('AI Comic Generator — Evaluation Report', sty['Title']),
        Paragraph('"The Misguided Enthusiasm of the Little Cactus Boyl" — Original Story', sty['Heading2']),
        Spacer(1, 0.15*inch),
        Paragraph('1. METHODOLOGY', sty['Heading1']),
        Paragraph(
            f'<b>Story:</b> Original by user — cactus boy learns self-acceptance<br/>'
            f'<b>Dialogue Model:</b> {d["model_dialogue"]}<br/>'
            f'<b>Image Model:</b> {d["model_image"]}<br/>'
            f'<b>Style:</b> Pure flat 2D vintage cartoon (no 3D, no realism)<br/>'
            f'<b>Evaluation:</b> {d["model_clip"]}<br/>'
            f'<b>AI Dialogues:</b> {d["llama_dialogue"]["generated"]}/{d["llama_dialogue"]["total"]} generated by Llama 3.1 8B',
            sty['Normal']
        ),
        Spacer(1, 0.12*inch),
        Paragraph('2. AUTOMATIC METRICS', sty['Heading1']),
    ]

    # === AUTOMATIC METRICS TABLE ===
    rows = [
        ['Metric', 'Value', 'Target', 'Status'],
        ['Avg LongCLIP',   f'{d["clip"]["avg"]:.2f}',    '>25', '✅' if d['clip']['avg']>=25 else '⚠️'],
        ['Min LongCLIP',   f'{d["clip"]["min"]:.2f}',    '≥20', '✅' if d['clip']['min']>=20 else '⚠️'],
        ['Excellent (≥30)',str(d['clip']['excellent']),  '—',   ''],
        ['Perplexity',     f'{d["perplexity"]:.2f}',     '<50', '✅' if d['perplexity']<50 else '⚠️'],
        ['Total Scenes',   str(d['scenes_count']),       '41',  '✅'],
        ['Llama Dialogues',f'{d["llama_dialogue"]["generated"]}/{d["llama_dialogue"]["total"]}', '—', ''],
    ]

    t = Table(rows, colWidths=[2.3*inch, 1.2*inch, 1.0*inch, 0.7*inch])
    t.setStyle(TableStyle([
        ('BACKGROUND', (0,0), (-1,0), colors.HexColor('#2D6A4F')),
        ('TEXTCOLOR',  (0,0), (-1,0), colors.white),
        ('FONTNAME',   (0,0), (-1,0), 'Helvetica-Bold'),
        ('FONTSIZE',   (0,0), (-1,-1), 10),
        ('GRID',       (0,0), (-1,-1), 0.8, colors.black),
        ('ROWBACKGROUNDS', (0,1), (-1,-1), [colors.white, colors.HexColor('#F0FFF4')]),
    ]))
    els += [t, Spacer(1, 0.15*inch)]

    # === HUMAN EVALUATIONS - 3 PARTS ===
    els += [
        Paragraph('3. HUMAN EVALUATION — ', sty['Heading1']),
        Paragraph(
            'The 41 scenes are evaluated by three human and was added manually here',
            sty['Normal']
        ),
        Spacer(1, 0.1*inch)
    ]

    # Generate table for each of the 3 evaluation parts
    for part_num, part_key, part_title in [
        (1, 'human_eval_1', 'first human'),
        (2, 'human_eval_2', 'second human'),
        (3, 'human_eval_3', 'third human')
    ]:
        he = d.get(part_key, {})

        # Section title
        els += [
            Spacer(1, 0.08*inch),
            Paragraph(f'<b>{part_title}</b>', sty['Heading3'])
        ]

        # Build table rows
        hr = [['Category', 'Rating', 'Notes']]
        for cat, val in he.items():
            if cat == 'scenes_range':
                continue
            elif cat == 'artifacts_count':
                count = val.get('count', '?')
                types = ', '.join(val.get('types', [])[:3])  # First 3 types
                if len(val.get('types', [])) > 3:
                    types += '...'
                hr.append(['Artifacts', f'Count: {count}', types or 'None listed'])
            else:
                sc = val.get('score')
                stars = ('★'*(sc or 0) + '☆'*(5-(sc or 0))) if sc else '___'
                notes = val.get('notes', '')[:45]
                display_name = cat.replace('_', ' ').title()
                hr.append([display_name, stars, notes])

        # Create and style table
        ht = Table(hr, colWidths=[2.2*inch, 1.0*inch, 3.5*inch])
        ht.setStyle(TableStyle([
            ('BACKGROUND', (0,0), (-1,0), colors.HexColor('#2D6A4F')),
            ('TEXTCOLOR',  (0,0), (-1,0), colors.white),
            ('FONTNAME',   (0,0), (-1,0), 'Helvetica-Bold'),
            ('FONTSIZE',   (0,0), (-1,-1), 9),
            ('GRID',       (0,0), (-1,-1), 0.8, colors.black),
            ('ROWBACKGROUNDS', (0,1), (-1,-1), [colors.white, colors.HexColor('#F0FFF4')]),
        ]))
        els += [ht]

    # Build PDF
    doc.build(els)
    return out

report_pdf = make_report(eval_data)
print(f'✅ Evaluation Report PDF generated with 3 human evaluation sections')
print(f'   Saved to: {report_pdf}')


# 📦 STEP 15 — SUBMISSION PACKAGE

In [ ]:
import shutil
sub = 'AI_Comic_Submission'
os.makedirs(f'{sub}/deliverables', exist_ok=True)
shutil.copy(comic_pdf,  f'{sub}/deliverables/1_Comic_Book.pdf')
shutil.copy(report_pdf, f'{sub}/deliverables/3_Evaluation_Report.pdf')
shutil.copy('comic/evaluation/results.json', f'{sub}/deliverables/eval_data.json')

groq_gen   = eval_data['llama_dialogue']['generated']
groq_total = eval_data['llama_dialogue']['total']
readme = f"""AI COMIC — SUBMISSION
=====================
Story      : The Misguided Enthusiasm of the Little Cactus Boy
Scenes     : {len(SCENES)} (8 acts)
Resolution : 1024x1024 (FLUX.1-dev)
Dialogue   : Llama 3.1 8B via Groq API + LangChain ({groq_gen}/{groq_total} generated)
CLIP model : LongCLIP zer0int/LongCLIP-GmP-ViT-L-14 (248-token limit)

RESULTS:
  Avg LongCLIP : {eval_data['clip']['avg']:.2f}
  Perplexity   : {eval_data['perplexity']:.2f}

PIPELINE:
  1. Llama 3.1 8B (Groq API) composes dialogue — 0 VRAM, instant
  2. FLUX.1-dev  renders 41 images — 2D  cartoon style
  3. Speech bubbles added (50px bold)
  4. LongCLIP evaluation (248-token limit, no truncation)
  5. GPT-2 perplexity on generated dialogues

TO SUBMIT:
  File → Download .ipynb → rename 2_Code.ipynb
  Place in {sub}/deliverables/
  Zip {sub}/ and submit
"""
with open(f'{sub}/README.txt','w') as f: f.write(readme)
print(readme)
print(f'✅ Submission ready: {sub}/')


# 🧪 STEP 16 — FULL VALIDATION

# ✅ PROJECT COMPLETE — Cactus Story

## Story: "The Misguided Enthusiasm of the Little Cactus Boy"

| Act | Scenes | Events |
|-----|--------|--------|
| Morning & Car Ride | 1–5 | Waking up, father drives, gives wise advice |
| School Arrival | 6–10 | Sees different plants, classrooms, bored in own class |
| Recess Drama | 11–14 | Goes to rain classroom, mocked for thorns, leaves sad |
| Monkey Orchid | 15–19 | Orchid warns him, he mocks her |
| Bad Decision | 20–25 | Removes thorns, joins rain class, gets sick |
| Walk Home & Attack | 26–31 | Father can't come, birds attack (no thorns!) |
| Rescue | 32–35 | Father rescues him from birds |
| Resolution | 36–41 | Confesses to father, learns lesson, happy ending |

---
## Pipeline

```
STEP 0B : Login — Groq API key + HuggingFace token (FLUX.1 gate)
STEP 6  : Llama 3.1 8B via Groq API + LangChain → 41 dialogues (0 VRAM)
STEP 7  : Load FLUX.1-dev (~33 GB VRAM)
STEP 8  : Generate 41 × 1024×1024 images
          · 20 steps · guidance_scale=3.5
          · Pure flat 2D cartoon style
          · NO negative_prompt (FLUX ignores it)
FREE B  : Free FLUX.1-dev
STEP 9  : Add 50px speech bubbles (Llama/Groq dialogue)
STEP 10 : Load LongCLIP (~1 GB VRAM)
STEP 10B: LongCLIP evaluation — context_length=248 enforced
FREE C  : Free LongCLIP
STEP 11 : GPT-2 perplexity
FREE D  : Free GPT-2
STEP 12 : Human evaluation
STEP 13 : Comic PDF
STEP 14 : Evaluation report PDF
STEP 15 : Submission package
```

## Model summary
| Role | Model | Notes |
|------|-------|-------|
| Dialogue | Llama 3.1 8B (Groq API) | Via LangChain, 0 VRAM |
| Images | FLUX.1-dev |  ~33 GB VRAM |
| Style | 2D flat cartoon | NO realism, NO 3D, NO photo |
| Evaluation | LongCLIP (context_length=248) | 248-token limit |
| Perplexity | GPT-2 | Story coherence |
